# GPA - Geometric Pose Affordance: 3D Human Pose with Scene Constraints

### Objective

__Add more data__

### Progress Log

- __Difference Between Test set and Validation set?__
- __How to add more data correctly?__
- __Standardization__

### Control Panel

### Import Libraries

In [1]:
import numpy as np

### Load Data

In [2]:
test_3d_gt_rel_std = np.load('test/test_3d_gt_rel_std.npy')
test_3d_gt_rel_mean = np.load('test/test_3d_gt_rel_mean.npy')
gt_3d_train_rel = np.load('train/gt_3d_train_rel.npy')
train_3d_gt_rel_std = np.load('train/train_3d_gt_rel_std.npy')
train_3d_gt_rel_mean = np.load('train/train_3d_gt_rel_mean.npy')
test_2d_gt_std = np.load('test/test_2d_gt_std.npy')
test_2d_gt_mean = np.load('test/test_2d_gt_mean.npy')
train_2d_gt_std = np.load('train/train_2d_gt_std.npy')
train_2d_gt_mean = np.load('train/train_2d_gt_mean.npy')
gt_2d_test = np.load('test/gt_2d_test.npy')
gt_2d_train = np.load('train/gt_2d_train.npy')
J_test_gt_mdp_enc2_15 = np.load('test/J_test_gt_mdp_enc2_15.npy')
J_train_gt_mdp_enc2_15 = np.load('train/J_train_gt_mdp_enc2_15.npy')
J_test_gt_mdp_15 = np.load('test/J_test_gt_mdp_15.npy')
J_train_gt_mdp_15 = np.load('train/J_train_gt_mdp_15.npy')
test_gt_mdp_15 = np.load('test/test_gt_mdp_15.npy')
train_gt_mdp_15 = np.load('train/train_gt_mdp_15.npy')
gt_2d_val = np.load('validation/gt_2d_val.npy')
gt_3d_val_rel = np.load('validation/gt_3d_val_rel.npy')

In [3]:
'''
print(gt_2d_train.shape)
print(J_train_gt_mdp_enc2_15.shape)
print(J_train_gt_mdp_15.shape)
print(train_gt_mdp_15.shape)
'''

'\nprint(gt_2d_train.shape)\nprint(J_train_gt_mdp_enc2_15.shape)\nprint(J_train_gt_mdp_15.shape)\nprint(train_gt_mdp_15.shape)\n'

### Define X

In [4]:
X_2d = np.reshape(gt_2d_train, (gt_2d_train.shape[0], 16 * 2))
X_enc2 = np.reshape(J_train_gt_mdp_enc2_15, (J_train_gt_mdp_enc2_15.shape[0], -1))
X_mdp15_J = np.reshape(J_train_gt_mdp_15, (J_train_gt_mdp_15.shape[0], -1))
X_mdp15 = np.reshape(train_gt_mdp_15, (train_gt_mdp_15.shape[0], -1))

'''
print(X_2d.shape)
print(X_enc2.shape)
print(X_mdp15_J.shape)
print(X_mdp15.shape)
'''

X = np.concatenate((X_2d, X_enc2, X_mdp15_J, X_mdp15), axis = 1)

### Define Y

In [5]:
Y = np.reshape(gt_3d_train_rel, (gt_3d_train_rel.shape[0], 15 * 3))

### Define Validation X

In [6]:
X_val = np.reshape(gt_2d_val, (gt_2d_val.shape[0], 16 * 2))

### Define Validation Y

In [7]:
Y_val = np.reshape(gt_3d_val_rel, (gt_3d_val_rel.shape[0], 15 * 3))

### Define Error Calculation Function

Error in 3D Euclidean Space between joint prediction and joint ground truth

In [8]:
def MPJPE_error(gt, pred): ## both prediction and ground truth in N*15*3, I will use this function to calculate final error and rank
    error = (gt - pred) ** 2
    pjpe = np.sum(error, 2) ** (1 / 2)
    mpjpe = np.mean(pjpe)
    return mpjpe

### Linear Regression Model

Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Linear Regression fits a linear model with coefficients w = (w1, ..., wp) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

regr_1 = linear_model.LinearRegression()
regr_1.fit(X, Y)
Y_val_pred_1 = regr_1.predict(X_val)
Y_val_pred_reshape_1 = np.reshape(Y_val_pred_1, (Y_val_pred_1.shape[0], 15, 3))
print(MPJPE_error(gt_3d_val_rel, Y_val_pred_reshape_1))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1442 is different from 32)

Best Error: __105.8613803487669__